# 用元类来注解类的属性

In [1]:
import logging

元类可以在某个类刚定义好，但是尚未使用的时候，提前修改或注解该类的属性。

**示例：**定义新的类，用来表示客户数据库里的某一行。

In [2]:
class Field(object):
    def __init__(self, name):
        self.name = name
        # 将实例的所有状态都作为protected字段
        self.internal_name = '_' + self.name

    def __get__(self, instance, instance_type):
        if instance is None: return self
        return getattr(instance, self.internal_name, '')

    def __set__(self, instance, value):
        # 存放在实例字典中
        setattr(instance, self.internal_name, value)

In [3]:
# 定义表示数据行的Customer类，为每个类属性指定对应的列名
class Customer(object):
    # Class attributes
    first_name = Field('first_name')
    last_name = Field('last_name')
    prefix = Field('prefix')
    suffix = Field('suffix')

In [4]:
foo = Customer()
print('Before:', repr(foo.first_name), foo.__dict__)
foo.first_name = 'Euclid'
print('After: ', repr(foo.first_name), foo.__dict__)

Before: '' {}
After:  'Euclid' {'_first_name': 'Euclid'}


**缺点：**写法重复。Field对象没有办法提前知道自己会赋给Customer类里的哪一个属性。

## 改进：使用元类进行处理。

In [5]:
class Meta(type):
    # 自动把相关属性设置好
    def __new__(meta, name, bases, class_dict):
        for key, value in class_dict.items():
            if isinstance(value, Field):
                value.name = key
                value.internal_name = '_' + key
        cls = type.__new__(meta, name, bases, class_dict)
        return cls

In [6]:
class DatabaseRow(object, metaclass=Meta):
    pass

In [7]:
class Field(object):
    def __init__(self):
        # These will be assigned by the metaclass.
        self.name = None
        self.internal_name = None
    def __get__(self, instance, instance_type):
        if instance is None: return self
        return getattr(instance, self.internal_name, '')

    def __set__(self, instance, value):
        setattr(instance, self.internal_name, value)

In [8]:
# 直接定义，不需要进行Field传参
class BetterCustomer(DatabaseRow):
    first_name = Field()
    last_name = Field()
    prefix = Field()
    suffix = Field()

In [9]:
foo = BetterCustomer()
print('Before:', repr(foo.first_name), foo.__dict__)
foo.first_name = 'Euler'
print('After: ', repr(foo.first_name), foo.__dict__)

Before: '' {}
After:  'Euler' {'_first_name': 'Euler'}
